### IPMOF - InterPenetrating Metal Organic Frameworks

#### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cut_off radius and grid size for energy map

#### Read structural information for MOF files in a given directory
1. Read MOF files in ".mol2" format from given directory and create a list
2. Create MOF objects for structure files
3. Initialize structural information for the MOFs

#### Read simulation input parameters
1. Read simulation parameters from input file

#### Calculate energy map
1. Determine packing amount of the MOF
2. Calculate packed coordinates for the base MOF
3. Calculate energy map

#### Start interpenetration
1. Energy map + mobile_mof

In [1]:
import math
import os
#os.chdir(r'/home/kutay/Documents/git/IPMOF/IPMOF_Python')
os.chdir(r'C:\Kutay\IPMOF\IPMOF_Python')
from forcefield import read_ff_parameters
from crystal import *
from energymap import *
from visualize import *
from interpenetration import *
from quaternion import *
%pylab inline

# Read excel file containing force field information
#excel_file_path = '/home/kutay/Documents/Research/FF_Parameters.xlsx'
excel_file_path = r'C:\Users\kutay\iPython\IPMOF\FF_Parameters.xlsx'
uff = read_ff_parameters(excel_file_path, 'uff')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [4]:
# Create list of MOFs
#mol2_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/mol2'
mol2_dir = r'C:\Kutay\MOFs\IPMOF_Python'
mol2_list = get_mof_list(mol2_dir, '.mol2')
print(mol2_list)

['KINFAQ.mol2', 'NUVWIL.mol2', 'OFODAP_clean.mol2', 'QIGBIR_clean.mol2', 'SAHYIK.mol2', 'UNIGEE_clean.mol2', 'ZIF-90.mol2']


In [8]:
# Read mol2 files and initialize MOF objects
base_mof = MOF()
base_mof.mol2_path = os.path.join(mol2_dir, mol2_list[4])
base_mof.initialize()
base_mof.initialize_ff(uff)
print('Base MOF selected as: ', base_mof.name)

mobile_mof = MOF()
mobile_mof.mol2_path = os.path.join(mol2_dir, mol2_list[4])
mobile_mof.initialize()
mobile_mof.initialize_ff(uff)
print('Mobile MOF selected as: ', mobile_mof.name)

Base MOF selected as:  SAHYIK
Mobile MOF selected as:  SAHYIK


In [9]:
cut_off = 12

base_mof.packing_factor = Packing.factor(base_mof.uc_size, cut_off)
uc_vectors = Packing.uc_vectors(base_mof.uc_size, base_mof.uc_angle)
trans_vec = Packing.translation_vectors(base_mof.packing_factor, uc_vectors)
base_mof.packed_coors = Packing.uc_coors(trans_vec, base_mof.packing_factor, uc_vectors, base_mof.atom_coors)
base_mof.edge_points = Packing.edge_points(uc_vectors)

print('Base MOF unit cell: ', base_mof.uc_size)
print('Packing factor:', base_mof.packing_factor)
print('Num of coor :', len(base_mof.packed_coors)*len(base_mof.packed_coors[0]))

Base MOF unit cell:  [25.669, 25.669, 25.669]
Packing factor: [3, 3, 3]
Num of coor : 11448


In [10]:
atom_list = get_uniq_atom_list([mobile_mof])
print('Calculating emap for', base_mof.name, 'with atoms:', atom_list['atom'])
emap = energy_map(base_mof, atom_list, cut_off, 1)

Calculating emap for SAHYIK with atoms: ['C', 'Zn', 'H', 'O']


In [11]:
emap_max = [emap[-1][0], emap[-1][1], emap[-1][2]]
emap_min = [emap[0][0], emap[0][1], emap[0][2]]

coor = [1.89, 11, 4.78]
emap_index = energy_map_index(coor, emap_max, emap_min)
atom_index = energy_map_atom_index('C', atom_list)

print('Energy map energy at ', emap[emap_index][:3], ' : ', emap[emap_index][atom_index] )
interpolation_energy = trilinear_interpolate(coor, atom_index, emap, emap_max, emap_min)
print('Interpolation energy at ', coor, ' : ', interpolation_energy)

Energy map energy at  [  2.  11.   5.]  :  -562.512156392
Interpolation energy at  [1.89, 11, 4.78]  :  -534.356103682


In [12]:
# Initialize simulation parameters
structure_energy_limit = 3E3
atom_energy_limit = 3E1
rotation_limit = 20
rotation_freedom = 30
summary_percent = 5

# Initialize simulation variables
Quat = Quaternion([0, 1, 1, 1])

initial_coors = initial_coordinates(base_mof, emap, atom_list, 3E10)
trial_limit = len(initial_coors) * rotation_limit
#omitted_coordinates = len(emap) - len(initial_coors)

abort_ip = False
structure_count = 0
structure_total_energy = 0
initial_coor_index = 0
# percent_complete = 0
div = round(trial_limit / (100 / summary_percent))
summary = {'percent':[], 'structure_count': [], 'trial_count':[]}
new_structures = []

sim_par = {'structure_energy_limit': structure_energy_limit,
           'atom_energy_limit': atom_energy_limit,
           'rotation_limit': rotation_limit,
           'rotation_freedom': rotation_freedom,
           'summary_percent': summary_percent
          }

In [14]:
def run_interpenetration(sim_par, base_mof, mobile_mof, emap, atom_list):
    """
    *** Not complete ***
    Run interpenetration algorithm with given simulation parameters and energy map.
    Returns simulation summary and structural information on the discovered structures.
    """
    # Initialize simulation parameters
    structure_energy_limit = sim_par['structure_energy_limit']
    atom_energy_limit = sim_par['atom_energy_limit']
    rotation_limit = sim_par['rotation_limit']
    rotation_freedom = sim_par['rotation_freedom']
    summary_percent = sim_par['summary_percent']

    # Initialize simulation variables
    Quat = Quaternion([0, 1, 1, 1])

    initial_coors = initial_coordinates(base_mof, emap, atom_list, atom_energy_limit)
    trial_limit = len(initial_coors) * rotation_limit
    rot_freedom = 360/rotation_freedom
    # omitted_coordinates = len(emap) - len(initial_coors)

    div = round(trial_limit / (100 / summary_percent))
    summary = {'percent': [], 'structure_count': [], 'trial_count': []}
    new_structures = []

    abort_ip = False
    structure_count = 0
    structure_total_energy = 0
    initial_coor_index = 0

    # Main interpenetration algorithm
    for t in range(trial_limit):  # Can iterate over something else???
        abort_ip = False
        # Interpenetration trial loop
        # Try interpenetration for a specific orientation by going through each atom in mobile mof
        for idx in range(len(mobile_mof)):    # Can iterate over something else???

            if not abort_ip:
                # If the interpenetration is just starting select rotation angles
                if idx == 0:
                    if t % rotation_limit == 0:
                        first_point = initial_coors[initial_coor_index]
                        initial_coor_index += 1
                        # initial_coor_trial_count += 1

                    # Determine random angles for rotation in 3D space
                    x_angle = 2 * math.pi * math.floor(rand() * rot_freedom) / rot_freedom
                    y_angle = 2 * math.pi * math.floor(rand() * rot_freedom) / rot_freedom
                    z_angle = 2 * math.pi * math.floor(rand() * rot_freedom) / rot_freedom

                    # Rotate first atom of the mobile MOF
                    atom_name = mobile_mof.atom_names[idx]
                    new_coor = Coor(mobile_mof.atom_coors[idx])
                    Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
                    # new_coor = Q.coor()
                    new_coor = Coor(Q.xyz())

                    translation_vector = first_point - new_coor  # Check if operation is correct

                    # Initialize new structure dictionay
                    structure = {'atom_names': [], 'atom_coors': [],
                                 'pbc_coors': [], 'energy': [], 'rotation': []}
                    structure['atom_coors'].append(first_point.xyz())  # Why first point not new_coor?
                    structure['pbc_coors'].append(new_coor.xyz())
                    structure['atom_names'].append(atom_name)
                    structure['rotation'] = [x_angle, y_angle, z_angle]

                # If interpenetration is still going on
                if idx < len(base_mof) and idx > 0:
                    atom_name = atom_name = mobile_mof.atom_names[idx]
                    new_coor = Coor(mobile_mof.atom_coors[idx])
                    Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
                    # new_coor = Q.coor()
                    new_coor = Coor(Q.xyz())

                    new_coor += translation_vector
                    pbc_coor = new_coor.pbc(base_mof.uc_size, base_mof.uc_angle, base_mof.frac_ucv)

                    emap_index = energy_map_index(pbc_coor.xyz(), emap_max, emap_min)
                    emap_atom_index = energy_map_atom_index(atom_name, atom_list)

                    point_energy = trilinear_interpolate(pbc_coor.xyz(), emap_atom_index, emap, emap_max, emap_min)
                    structure_total_energy += point_energy

                    if structure_total_energy > structure_energy_limit:
                        structure_total_energy = 0
                        abort_ip = True
                        break  # Fix this part (break interpenetration trial loop)
                    elif point_energy > atom_energy_limit:
                        structure_total_energy = 0
                        abort_ip = True
                        break  # Fix this part (break interpenetration trial loop)
                    else:
                        structure['atom_coors'].append(new_coor.xyz())
                        structure['pbc_coors'].append(pbc_coor.xyz())
                        structure['atom_names'].append(atom_name)

                # If interpenetration trial ended with no collision
                if idx == len(mobile_mof) - 1:
                    # Record structure information!!!!
                    structure['energy'] = structure_total_energy
                    new_structures.append(structure)
                    structure_count += 1
                    structure_total_energy = 0

        # Record simulation progress according to division (div)
        if t % div == 0:
            percent_complete = round(t / trial_limit * 100)

            # Record summary information
            summary['percent'].append(percent_complete)
            summary['structure_count'].append(structure_count)
            summary['trial_count'].append(t)

    return summary, new_structures

In [15]:
summary, new_structures = run_interpenetration(sim_par, base_mof, mobile_mof, emap, atom_list)

In [16]:
# Get minimum energy structure by sorting total structure energies
min_energy_structure = sorted(new_structures, key=lambda k: k['energy'])[0]

In [17]:
def join_structures(base_mof, new_structure):
    joined_atom_names = []
    joined_atom_coors = []
    for atom, coor in zip(new_structure['atom_names'], new_structure['atom_coors']):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    for atom, coor in zip(base_mof.atom_names, base_mof.atom_coors):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    return joined_atom_names, joined_atom_coors

# Define file name and export directory
file_name = base_mof.name + '_' + mobile_mof.name
export_dir = r'C:\Kutay\MOFs\IPMOF_Python\export'
#export_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/export'

# Join interpenetrating structures
joined_names, joined_coors = join_structures(base_mof, min_energy_structure)

# Export coordinates in .xyz format
export_xyz(joined_coors, joined_names, file_name, export_dir)

In [20]:
rotation_info = min_energy_structure['rotation']
ext_cut_off = 50
energy_limit = sim_par['atom_energy_limit']

In [31]:
def check_extension(base_MOF, mobile_MOF, rotation_info, emap, emap_atom_list, energy_limit, ext_cut_off):
    """
    *** Not Complete ***
    Checks collision between interpenetrating layer and base layer for a determined distance.
    Distance is calculated from given ext_cut_off value which determines the packing amount of the
    interpenetrating layer.
    Each coordinate in the interpenetrating layer is checked for high energy values by applying
    perodic boundary conditions to the coordinate according to energy map of the base layer.
    """
    emap_max = [emap[-1][0], emap[-1][1], emap[-1][2]]
    emap_min = [emap[0][0], emap[0][1], emap[0][2]]

    packing_factor = Packing.factor(mobile_MOF.uc_size, ext_cut_off)
    uc_vectors = Packing.uc_vectors(mobile_MOF.uc_size, mobile_MOF.uc_angle)
    trans_vec = Packing.translation_vectors(packing_factor, uc_vectors)
    packed_coors = Packing.uc_coors(trans_vec, packing_factor, uc_vectors, mobile_MOF.atom_coors)

    #rotated_packed_coors = rotate_unit_cell(packed_coor, rotation_info)
    x_angle = rotation_info[0]
    y_angle = rotation_info[1]
    z_angle = rotation_info[2]

    collision = False
    for unit_cell in packed_coors:
        if not collision:
            for coor in unit_cell:

                if not collision:

                    new_coor = Coor(coor)
                    Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
                    new_coor = Coor(Q.xyz())

                    pbc_coor = new_coor.pbc(base_MOF.uc_size, base_MOF.uc_angle, base_MOF.frac_ucv)

                    emap_index = energy_map_index(pbc_coor, emap_max, emap_min)
                    atom_index = energy_map_atom_index(atom_name, emap_atom_list)

                    energy = emap[emap_index][atom_index]
                    if energy < energy_limit:
                        continue
                    else:
                        collision = True
                        break
                else:
                    break
        else:
            break

    return collision

In [32]:
collision = check_extension(base_mof, mobile_mof, rotation_info, emap, atom_list, energy_limit, ext_cut_off)

TypeError: zip argument #1 must support iteration

In [28]:
rotation_info[0]

2.6179938779914944